#Notebook de importação do historico

In [0]:
from pyspark.sql.functions import *

In [0]:
spark.conf.set(
    "fs.azure.account.key.sadatalake002poc2022.dfs.core.windows.net",
    ""
)

In [0]:
dbutils.fs.ls("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw")

Out[5]: [FileInfo(path='abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/departments.csv', name='departments.csv', size=200, modificationTime=1672081695000),
 FileInfo(path='abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/hired_employees.csv', name='hired_employees.csv', size=94932, modificationTime=1672081695000),
 FileInfo(path='abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/jobs.csv', name='jobs.csv', size=4601, modificationTime=1672081695000)]

In [0]:
df_departments = spark.read.csv("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/departments.csv")
display(df_departments)

_c0,_c1
1,Product Management
2,Sales
3,Research and Development
4,Business Development
5,Engineering
6,Human Resources
7,Services
8,Support
9,Marketing
10,Training


In [0]:
df_departments = (df_departments
                          .withColumnRenamed('_c0','id')
                          .withColumnRenamed('_c1','departments')                  
                 )
display(df_departments)

id,departments
1,Product Management
2,Sales
3,Research and Development
4,Business Development
5,Engineering
6,Human Resources
7,Services
8,Support
9,Marketing
10,Training


In [0]:
df_hemployees = spark.read.csv("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/hired_employees.csv")
display(df_hemployees)

_c0,_c1,_c2,_c3,_c4
1,Harold Vogt,2021-11-07T02:48:42Z,2,96
2,Ty Hofer,2021-05-30T05:43:46Z,8,null
3,Lyman Hadye,2021-09-01T23:27:38Z,5,52
4,Lotti Crowthe,2021-10-01T13:04:21Z,12,71
5,Gretna Lording,2021-10-10T22:22:17Z,6,80
6,Marlow Antecki,2021-04-23T23:45:42Z,6,95
7,Joan Rillett,2021-10-10T01:33:31Z,9,78
8,Ulrick Nucciotti,2021-07-24T01:28:40Z,8,169
9,Lucretia Northcote,2021-04-01T21:22:47Z,9,8
10,Arty Giacobo,2022-02-08T12:27:07Z,6,62


In [0]:
df_hemployees = (df_hemployees
                    .withColumnRenamed('_c0','id')
                    .withColumnRenamed('_c1','name')
                    .withColumnRenamed('_c2','datetime')
                    .withColumnRenamed('_c3','department_id')
                    .withColumnRenamed('_c4','job_id')
                )
display(df_hemployees)

id,name,datetime,department_id,job_id
1,Harold Vogt,2021-11-07T02:48:42Z,2,96
2,Ty Hofer,2021-05-30T05:43:46Z,8,null
3,Lyman Hadye,2021-09-01T23:27:38Z,5,52
4,Lotti Crowthe,2021-10-01T13:04:21Z,12,71
5,Gretna Lording,2021-10-10T22:22:17Z,6,80
6,Marlow Antecki,2021-04-23T23:45:42Z,6,95
7,Joan Rillett,2021-10-10T01:33:31Z,9,78
8,Ulrick Nucciotti,2021-07-24T01:28:40Z,8,169
9,Lucretia Northcote,2021-04-01T21:22:47Z,9,8
10,Arty Giacobo,2022-02-08T12:27:07Z,6,62


In [0]:
df_jobs = spark.read.csv("abfss://poc@sadatalake002poc2022.dfs.core.windows.net/raw/jobs.csv")
display(df_jobs)

_c0,_c1
1,Marketing Assistant
2,VP Sales
3,Biostatistician IV
4,Account Representative II
5,VP Marketing
6,Environmental Specialist
7,Software Consultant
8,Office Assistant III
9,Information Systems Manager
10,Desktop Support Technician


In [0]:
df_jobs = (df_jobs
              .withColumnRenamed('_c0','id')
              .withColumnRenamed('_c1','job')
          )
display(df_jobs)

id,job
1,Marketing Assistant
2,VP Sales
3,Biostatistician IV
4,Account Representative II
5,VP Marketing
6,Environmental Specialist
7,Software Consultant
8,Office Assistant III
9,Information Systems Manager
10,Desktop Support Technician


In [0]:
jdbcUser = "sqladmin"
jdbcPassword  = ""
jdbcHostname = "serverpoc2022.database.windows.net"
jdbcDatabase = "poc2022 "
jdbcPort = 1433
jdbcDriver = "com.microsoft.sqlserver.jdbc.SQLServerDriver"

jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
  "user" : jdbcUser,
  "password" : jdbcPassword,
  "driver" : jdbcDriver
}

In [0]:
table = "tb_jobs"

df_jobs.write.mode("overwrite") \
      .format("jdbc") \
      .option("url", jdbcUrl) \
      .option("dbtable", table) \
      .option("user", jdbcUser) \
      .option("password", jdbcPassword) \
      .option("driver", jdbcDriver) \
      .save()

In [0]:
spark.read.jdbc(url=jdbcUrl,table="tb_jobs",properties=connectionProperties).show()

+---+--------------------+
| id|                 job|
+---+--------------------+
|  1| Marketing Assistant|
|  2|            VP Sales|
|  3|  Biostatistician IV|
|  4|Account Represent...|
|  5|        VP Marketing|
|  6|Environmental Spe...|
|  7| Software Consultant|
|  8|Office Assistant III|
|  9|Information Syste...|
| 10|Desktop Support T...|
| 11|   Financial Advisor|
| 12|Computer Systems ...|
| 13|Automation Specia...|
| 14|Help Desk Technician|
| 15| Office Assistant II|
| 16|  VP Quality Control|
| 17| Office Assistant IV|
| 18|   Financial Analyst|
| 19| Electrical Engineer|
| 20|   Chemical Engineer|
+---+--------------------+
only showing top 20 rows

